## Using CNN on spectrogram images of audio.

In [60]:
import tensorflow as tf

from scipy.io.wavfile import read, write
import playsound
import os 
import librosa
import librosa.display
import librosa.feature
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import pandas as pd
from pydub import AudioSegment
from os import path

#from PIL import Image
import pathlib
import csv 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import keras
from keras import layers
from keras import layers
import keras
from keras.models import Sequential
import warnings

from keras.preprocessing.image import ImageDataGenerator

from keras import layers
from keras.layers import Activation, Dense, Dropout, Conv2D, Flatten, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling1D, AveragePooling2D, Input, Add
from keras.models import Sequential
from keras.optimizers import SGD

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [58]:
import split_folders
split_folders.ratio('data/newImage/', output="./data1", seed=1337, ratio=(.8, .2)) # default values


Copying files: 0 files [00:00, ? files/s]
Copying files: 63 files [00:00, 624.59 files/s]
Copying files: 133 files [00:00, 643.54 files/s]
Copying files: 196 files [00:00, 638.62 files/s]
Copying files: 256 files [00:00, 624.94 files/s]
Copying files: 311 files [00:00, 598.45 files/s]
Copying files: 361 files [00:00, 556.17 files/s]
Copying files: 411 files [00:00, 486.74 files/s]
Copying files: 457 files [00:00, 477.31 files/s]
Copying files: 503 files [00:00, 467.99 files/s]
Copying files: 549 files [00:01, 439.31 files/s]
Copying files: 593 files [00:01, 429.59 files/s]
Copying files: 636 files [00:01, 412.55 files/s]
Copying files: 678 files [00:01, 410.63 files/s]
Copying files: 719 files [00:01, 404.71 files/s]
Copying files: 760 files [00:01, 392.62 files/s]
Copying files: 800 files [00:01, 390.50 files/s]
Copying files: 840 files [00:01, 351.22 files/s]
Copying files: 880 files [00:01, 361.86 files/s]
Copying files: 917 files [00:02, 360.29 files/s]
Copying files: 954 files [0

In [64]:
train_datagen = ImageDataGenerator(
rescale=1./255, # rescale all pixel values from 0-255, so aftre this step all our pixel values are in range (0,1)
shear_range=0.2, #to apply some random tranfromations
zoom_range=0.2, #to apply zoom
horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory('data1/train', target_size=(64, 64), batch_size=32, class_mode='categorical', shuffle = True)

test_set = test_datagen.flow_from_directory('data1/val', target_size=(64, 64), batch_size=32, class_mode='categorical', shuffle = True )

test_actual = test_datagen.flow_from_directory('data1/Test Images', target_size=(64, 64), batch_size=32, class_mode='categorical', shuffle = False )

Found 1920 images belonging to 6 classes.
Found 480 images belonging to 6 classes.
Found 1200 images belonging to 1 classes.


## CNN

In [65]:
resFile = 'CNN'
model = Sequential()
input_shape=(64, 64, 3)

# 1st hidden layer
model.add(Conv2D(32, (3, 3), strides=(2, 2), input_shape=input_shape))
model.add(AveragePooling2D((2, 2), strides=(2,2)))
model.add(Activation('relu'))

# 2nd hidden layer
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(AveragePooling2D((2, 2), strides=(2,2)))
model.add(Activation('relu'))

# 3rd hidden layer
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(AveragePooling2D((2, 2), strides=(2,2)))
model.add(Activation('relu'))

# Flatten
model.add(Flatten())
model.add(Dropout(rate=0.5))

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(rate=0.5))

#Output layer
model.add(Dense(6))
model.add(Activation('softmax'))
model.summary()

print(len(training_set), len(test_set), type(training_set))
epochs = 200
batch_size = 8
learning_rate = 0.01
decay_rate = learning_rate / epochs
momentum = 0.9
sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
model.compile(optimizer="sgd", loss="categorical_crossentropy", metrics=['accuracy'])

model.fit_generator(training_set, steps_per_epoch=100, epochs=100, validation_data=test_set, validation_steps=200)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 31, 31, 32)        896       
_________________________________________________________________
average_pooling2d_4 (Average (None, 15, 15, 32)        0         
_________________________________________________________________
activation_6 (Activation)    (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 15, 15, 64)        18496     
_________________________________________________________________
average_pooling2d_5 (Average (None, 7, 7, 64)          0         
_________________________________________________________________
activation_7 (Activation)    (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 7, 7, 64)         

Epoch 95/100
100/100 [==============================] - 92s 918ms/step - loss: 1.5579 - accuracy: 0.3653 - val_loss: 1.5691 - val_accuracy: 0.3775
Epoch 96/100
100/100 [==============================] - 95s 955ms/step - loss: 1.5456 - accuracy: 0.3697 - val_loss: 1.6622 - val_accuracy: 0.3953
Epoch 97/100
100/100 [==============================] - 93s 933ms/step - loss: 1.5510 - accuracy: 0.3691 - val_loss: 1.5932 - val_accuracy: 0.3878
Epoch 98/100
100/100 [==============================] - 94s 939ms/step - loss: 1.5515 - accuracy: 0.3725 - val_loss: 1.4966 - val_accuracy: 0.3819
Epoch 99/100
100/100 [==============================] - 100s 997ms/step - loss: 1.5467 - accuracy: 0.3616 - val_loss: 1.5193 - val_accuracy: 0.3716
Epoch 100/100
100/100 [==============================] - 97s 966ms/step - loss: 1.5505 - accuracy: 0.3706 - val_loss: 1.5598 - val_accuracy: 0.3584


## Write to file

In [67]:
predicted = model.predict_generator(test_actual, steps=100, verbose=1)
predicted_class_indices=np.argmax(predicted,axis=1)

labels = (training_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
#predictions = predictions[:1200]
df = pd.read_csv('data/test_idx.csv')

i = 0
final = []
for index, row in df.iterrows():
    row1 = [row[0], predictions[i]]
    final.append(row1)
    i += 1

df2 = pd.DataFrame(data = final, columns = ['id', 'genre'])
df2.to_csv('data/fin_CNN.csv', index = False)
print('Done.')

100/100 [==============================] - 25s 245ms/step
Done.


In [66]:
li = [model]
np.save('data/CNN_newImage',li)

In [3]:
li = np.load('CNN_newImage.npy', allow_pickle = True)
model = li[0]

In [16]:
metric = model.evaluate_generator(generator=test_set,
steps=100)
print('Accuracy: ', metric[1])

Accuracy:  0.4593749940395355
